In [1]:
from models import FCC, Siamese, Siamese_no_sharing
import dlc_practical_prologue
import torch
from torch import nn
from torch import optim
import torchvision
from torch.nn import functional as F
from tqdm import trange
import matplotlib.pyplot as plt
import models
from utils import Proj1

In [2]:
# Load data
train_input, train_target, train_classes, test_input, test_target, test_classes = dlc_practical_prologue.generate_pair_sets(1000)

train_data = (train_input, train_target, train_classes)
test_data = (test_input, test_target, test_classes)

# INstantiate models
models = (FCC(), FCC(), Siamese_no_sharing(), Siamese_no_sharing(), Siamese(), Siamese())
models[1].setAuxillary()
models[3].setAuxillary()
models[5].setAuxillary()
model_names = ('FCC' , 'FCC w Auxiliary Loss','Siamese (no weight sharing)', 'Siamese (no weight sharing and Auxiliary loss)', 'Siamese (weight sharing)', 'Siamese (weight sharing and Auxiliary Loss)')

In [3]:
def evaluate_all(models, model_names, train_data, test_data, batch_size=100, rounds=10, epochs=25):
    '''
    - train and test_date should be complete (train_input, train_target, train_classes)
    - model_names are strings, that should contain the word Auxiliary
    '''
    # Unpack data
    train_input, train_target, train_classes = train_data
    test_input, test_target, test_classes = test_data
    # Initialize err rates arrays
    train_error_rates = torch.empty(rounds)
    test_error_rates = torch.empty(rounds)
    
    # Iterate models
    for model, name in zip(models, model_names):
        print(f'Training model {name}:')
        # Iterate over r to get average
        for r in trange(rounds):
            # Call train_model with the correct parameters
            if 'Aux' in name:
                Proj1.train_model(model, train_input, train_target, batch_size, epochs, train_classes)
            else:
                Proj1.train_model(model, train_input, train_target, batch_size, epochs, train_classes=None)
            # Get error on train and test set
            train_error_rates[r] = Proj1.compute_nb_errors(model, train_input, train_target, batch_size)/train_input.size(0)
            test_error_rates[r] = Proj1.compute_nb_errors(model, test_input, test_target, batch_size)/test_input.size(0)
            
        #print(f'For the model {name}, the train average error rate is {train_error_rates.mean()} and the test average error rate {test_error_rates.mean()}\n')
        print('For the model {}, the train average error rate is {:.3}% and the test average error rate is {:.3}%.\n'
              .format(name, 100*train_error_rates.mean(), 100*test_error_rates.mean()))
#     return error_rates


# def error_rates(model, input_data, target_data, batch_size=100, rounds=10, epochs=25, target_classes=None):
#     error_rates = torch.empty(rounds)
#     for r in trange(rounds):
#         train_model(model, train_input, train_target, 100, 25, target_classes)
#         error_rates[r] = compute_nb_errors(model, test_input, test_target, 100)/test_input.size(0)
#     return error_rates

In [4]:
evaluate_all(models, model_names, train_data, test_data, batch_size=100, rounds=1, epochs=25)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Training model FCC:


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

For the model FCC, the train average error rate is 0.2% and the test average error rate is 14.4%.

Training model FCC w Auxiliary Loss:


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

...

For the model Siamese (no weight sharing and Auxiliary loss), the train average error rate is 11.2% and the test average error rate is 15.3%.

Training model Siamese (weight sharing):


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

For the model Siamese (weight sharing), the train average error rate is 11.7% and the test average error rate is 14.7%.

Training model Siamese (weight sharing and Auxiliary Loss):


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.02s/it]

For the model Siamese (weight sharing and Auxiliary Loss), the train average error rate is 9.0% and the test average error rate is 10.1%.



In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [8]:
for model in models:
    print(count_parameters(model))

241876
241876
27828
27828
14196
14196
